# Statistical analysis
Questions => are fus1 different from 0 in early timepoints: t-test

=> trend of all proteins : linear regression +t-test 
 
 => are exocyst proteins getting farther away from Myo52 centroid signal : linear regression + t-test
 
 => do two proteins have the same trajectories? permutation test
          
To discuss with Frederic => consider each point individually : clustering => are all points in same group found in same cluster? 
PCA on individual POI/ all => composante variability ? 

NOTE: either prepare .csv file here and export towards R for analysis or perform analysis directly in python (linear regression + t-test) 

In [1]:
#general infos considering experiement 
#files containing fusion events folder
path= '/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/' 
files = ['Prm1','Dni1','Dni2','Fus1','Exo70', 'Myo51', 'Cdc8','Exo84']
Myo= ['Myo52_M', 'Myo52_P']
RefM= '_B'
RefP= '_R'
colPOI=['lime', 'darkturquoise','hotpink','forestgreen', 'purple', 'darkorange','goldenrod','darkslateblue']
colMyo=['darkblue','red']

In [2]:
# import all necessary packages 
from trajalign.traj import Traj
from trajalign.average import load_directory
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import norm as norm


## datasets preparation (timepoints vs d_PPOI distances)

load directed_distances traj files
Linear regression => consider all points of the array 
 => 2 linear regression on fus1? 
 Mean comparison => bin data in 2 groups : early fusion (t=[,] )/ late fusion (t= [,0[

column: timepoints + n° Replicats 
timepoints => np.linspace(-60,0,21)


In [3]:
#load directed distances trajectories files 

def replicats_nb(POI):
    rep_list = [ folder for folder in os.listdir(path+POI+'/aligned_trajectories/') if 'Rep_' in folder ]
    r= len(rep_list)
    return r

def get_data(POI,rep) :
    directory_rep = os.listdir( path+POI+'/aligned_trajectories/Rep_'+str(rep+1)+'/') #absolute path of directory to charge data from
    f = [ folder for folder in directory_rep if 'cell_' in folder ] # 'cell_' -> folder nam to list
    return f 

n= len(files) 
Rep= []
for i in range(n):
    r=replicats_nb(files[i])
    Rep.append(r)


folders=[[] for _ in range(n)] #lists for POI rep folders => 1
Cells=[[] for _ in range(n)] #lists of trajectories for each events for each POI of each replicate

for i in range(len(Rep)):
    folders[i]=[[] for _ in range(Rep[i])] 
    Cells[i]=[[] for _ in range(Rep[i])] 

# list all cell_ folders for each POI replicats

for i in range(n) :
    for r in range(Rep[i]):
        f=get_data(files[i],r)
        folders[i][r]= f 
        
ct=0
for f in folders:
    r=1
    i=0 
    for rep in f:
        for event in rep: 
            t_l = load_directory(path+files[ct] +'/aligned_trajectories/Rep_'+str(r)+'/'+ event ,pattern= 'dir_dist', comment_char = '#', t_unit='min' , frames = 0 ,t= 1 , f = 2 ,f_unit = 'nm' )
            Cells[ct][i].extend(t_l)
        r+=1
        i+=1
    ct+=1
    

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applie

dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectori


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt


dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectori


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt


...


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt


dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<



dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectori

dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<



dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

dir_dist_M_POI.txt
dir_dist_P_M.txt
dir_dist_P_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<



In [4]:
# order trajectories per list according to measured distance
vect_d_P_POI= [[] for _ in range(n)] # directed distance from Myo52_P towards POI for each events
vect_d_P_M= [[] for _ in range(n)]   # directed distance from Myo52_P towards Myo52_M for each events
vect_d_M_POI= [[] for _ in range(n)]  #  directed distance from Myo52_M towards POI for each events 
for i in range(len(Rep)):
    vect_d_P_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_M_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_P_M[i]= [[] for _ in range(Rep[i])]
    
for i in range(n):
        for r in range (len(Cells[i])):
            for t in Cells[i][r]:
                if "P_POI" in t.annotations() ['file'] : 
                    vect_d_P_POI[i][r].append(t)
                elif "M_POI" in t.annotations() ['file'] :
                    vect_d_M_POI[i][r].append(t)
                else : 
                    vect_d_P_M[i][r].append(t)
                    

### Generation CSV files to import in R

timepopints -60 to 0, include Event id, Protein name, Replicat number and type of distance measured

In [5]:
#create empty dataframe with right colums

def dist_dataframe (dist_list,path,POI,folder, what='P_M') : 
    data=[[] for _ in range(len(dist_list))]
    #creating list of colums names
    colnames=['Event','Protein','Replicat','Dist'] # infos about the fusion event, event = cell folder id, Dist=directed distance measured
    tp=np.linspace(-60,0,21) # to add timepoints as strings
    for i in range(len(tp)):
        t=str(tp[i])
        colnames.append(t)
        
    for i in range (len(dist_list)):
        matrice={}
        for k in range (len (colnames)):
            matrice[colnames[k]] = []   
        for r in range(len(dist_list[i])):
            for ev in range(len (dist_list[i][r])):
                matrice['Event'].append(folder[i][r][ev])
                matrice['Protein'].append(POI[i])
                matrice['Replicat'].append(str(r+1))
                matrice['Dist'].append(what)
                
                for t in tp:
                    test='False'
                    for ct in range (len (dist_list[i][r][ev])) :
                        if (dist_list[i][r][ev].t(ct)==t):
                            matrice[str(t)].append(dist_list[i][r][ev].f(ct))
                            test= 'TRUE'
                    if (test!='TRUE'):
                        matrice[str(t)].append(float('Nan'))         
                        
        df=pd.DataFrame(matrice)
        df=df.set_index('Event')   
        os.makedirs(path+POI[i]+'/aligned_trajectories/Data/', exist_ok=True)
        df.to_csv(path+POI[i]+'/aligned_trajectories/Data/dir_distance_%s.csv'%what)
        data[i].append(matrice)
    return data

In [6]:
df_PPOI=dist_dataframe(vect_d_P_POI,path,files,folders, what='P_POI')
df_MPOI=dist_dataframe(vect_d_M_POI,path,files,folders, what='M_POI')
df_PM=dist_dataframe(vect_d_P_M,path,files,folders, what='P_M')

## Linear regression models 

On all points, uses stats.model package. NOTE => work on residual

from IPython.display import IFrame
IFrame("./samples/simple3.pdf", width=600, height=300) => to try getting images to show up

In [15]:
#what argument available: scatter: on list with all events
def linear_regression (list, type, name,r, what= 'scatter'): #list= list of directed distances for each timepoint of each event of one POI -> the one used for averaging
    if what=='scatter' :
        x=[]
        x_plt=[]
        y=[]
        for traj in list:
            for ct in range(len(traj)):
                if (-60<traj.t(ct) <0):
                    if (np.isnan(traj.f(ct)) == False):
                        x.append(traj.t(ct))
                        x_plt.append(traj.t(ct))
                        y.append(traj.f(ct))
                        
    x = np.array(x).reshape ((-1,1))
    x_plt=np.array(x_plt)
    y= np.array (y)
    x = sm.add_constant(x)   
    model=sm.OLS(y, x).fit()
    fig=plt.figure(figsize=(25,18))
    fig=sm.graphics.plot_regress_exog(model, 'x1', fig=fig)
    fig.tight_layout(pad=2.0)
    plt.savefig(path+name+'/aligned_trajectories/Data/residual_plot_%s_%s_Rep_%s.pdf'%(type,name,str(r+1)))
    plt.close(fig)    

    return x,x_plt,y, model



#list_dist: all points events, av= average directed distance, type = which distance is considered
#what parameters: scatter, scatter_ecc (for eccentricity)

def plot_regression (list_dist,names,color,type, what = 'scatter'):
    x=[[] for _ in range(n)]
    x_plt=[[] for _ in range(n)]
    y=[[] for _ in range(n)]
    results=[[] for _ in range(n)]
    r_sq=[[] for _ in range(n)]
    coefs= [[] for _ in range(n)]
    fx=[[] for _ in range(n)]
    list=[[] for _ in range(n)]  
    for k in range (len(Rep)):
        x[k]=[[] for _ in range(Rep[k])]
        x_plt[k]=[[] for _ in range(Rep[k])]
        y[k]=[[] for _ in range(Rep[k])]
        results[k]=[[] for _ in range(Rep[k])]
        r_sq[k]=[[] for _ in range(Rep[k])]
        coefs[k]= [[] for _ in range(Rep[k])]
        fx[k]=[[] for _ in range(Rep[k])]
        list[k]=[[] for _ in range(Rep[k])] 
    
    if what == 'scatter':
            for i in range(len(list_dist)): 
                for r in range (len (list_dist[i])):
                    x[i][r], x_plt[i][r],y[i][r], results[i][r]= linear_regression(list_dist[i][r],type, names[i],r, what= 'scatter')
                    r_sq[i][r]= results[i][r].rsquared   #. rsquared return R²
                    coefs[i][r]= results[i][r].params    # get coefficients list
                    fx[i][r]=results[i][r].fittedvalues
                    list[i][r]=[np.absolute(results[i][r].params[1]),results[i][r].bse[1]] 
                            
                    with open(path+names[i]+'/aligned_trajectories/Data/%s_%s_Rep_%s.txt'%(type,names[i],str(r+1)),'w') as txt:
                        print(results[i][r].summary(),file=txt)
                        print(results[i][r].params,file=txt)
                        print(results[i][r].pvalues, file=txt)
               
                   
    if what == 'scatter_ecc':
            for i in range(len(list_dist)): 
                   for r in range (len (list_dist[i])):
                    x[i][r], x_plt[i][r],y[i][r], results[i][r]= linear_regression(list_dist[i][r], type,names[i],r, what= 'scatter')
                    r_sq[i][r]= results[i][r].rsquared
                    coefs[i][r]= results[i][r].params
                    fx[i][r]=results[i][r].fittedvalues
                    list[i][r]= [np.absolute(results[i][r].params[1]),results[i][r].bse[1]]
               
                    with open(path+names[i]+'/aligned_trajectories/Data/%s_%s_Rep_%s.txt'%(type,names[i],str(r+1)),'w') as txt:
                        print(results[i][r].summary(),file=txt)
                        print(results[i][r].params, file=txt)
                        print(results[i][r].pvalues, file=txt)
                     
    plt.figure(figsize=((20*3,15*9)))  
    for i in range (len(list_dist)):  #nb of pd pages (ideally)
        for r in range (len(list_dist[i])):
            plt.subplot (6,3,r+1)
            plt.xlim (-60, 0)
            plt.ylim(fx[i][r].min()-100, fx[i][r].max()+100)
            plt.grid()
            if what == 'scatter':
                plt.scatter (x_plt[i][r],y[i][r], color=color[i], alpha= 0.3)
            if what == 'scatter_ecc':
                plt.scatter (x_plt[i][r],y[i][r], color=color[i], alpha= 0.3)
                plt.ylim(1, 3)
       
            plt.plot(x_plt[i][r], fx[i][r], color= 'blue', alpha=1)
            plt.hlines(0,-60,0, color='red')
            plt.text(-55, fx[i][r].max()+50,r'f(x)=%s + %s * x'%(round(coefs[i][r][0],2),round(coefs[i][r][1],4)), fontsize = 35)
            plt.text(-15, fx[i][r].max()+50, r'R²=%s'%(round(r_sq[i][r],2)), fontsize = 35)
            plt.title(names[i], fontsize =45) 
            plt.xlabel('time (min)', fontsize= 40)
            plt.ylabel('vectorial distance %s'%type, fontsize= 40)
        plt.savefig(path+names[i]+'/aligned_trajectories/Data/StatsM_Linear_regression_%s.pdf'%type)
        plt.close()      
    return list  

In [16]:
LR_P_POI= [[] for _ in range(n)]
LR_P_M= [[] for _ in range(n)]
for i in range(len(Rep)):
    LR_P_POI[i]=[[] for _ in range(Rep[i])] 
    LR_P_M[i]=[[] for _ in range(Rep[i])] 
    
    
LR_P_POI=plot_regression (vect_d_P_POI,files,colPOI,'P_POI', what = 'scatter')
LR_P_M=plot_regression (vect_d_P_M, files ,colPOI,'Myo52_P_M', what = 'scatter')

#Note: code will have to include Rep somehow => save in Rep files instead data one
#notes => try plot all replicats on same and save all POI in multipage pdf

eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1
eval_env: 1


In [ ]:
#notes => try plot all replicats on same and save all POI in multipage pdf